In [10]:
import yfinance as yf
import pandas as pd

In [14]:
data = yf.download("PBW", start="2010-01-01", end="2022-03-19")

[*********************100%***********************]  1 of 1 completed


In [15]:
df = pd.DataFrame(data)

In [16]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,55.450001,56.750000,55.450001,56.750000,43.996807,107960
2010-01-05,56.849998,57.750000,56.450001,57.400002,44.500721,101740
2010-01-06,57.500000,58.549999,57.450001,58.250000,45.159714,181180
2010-01-07,58.250000,58.400002,57.700001,58.099998,45.043419,97420
2010-01-08,58.000000,58.799999,58.000000,58.799999,45.586124,75240
